<a href="https://colab.research.google.com/github/vladislavlisin/Stylometry_Extra/blob/main/Stylometry_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Смотрим на данные**

In [ ]:
# работаем в google colab
# подключаем гугл диск с данными
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np

corpus = pd.read_csv("/content/drive/MyDrive/poems_ok.csv", delimiter=',', encoding='utf-8')

corpus.head()

,index,author,name,year,text
0,0,Пушкин,NaN,0,"Когда сожмешь ты снова руку,\nКоторая тебе дар..."
1,1,Пушкин,NaN,0,"Тебя ль я видел, милый друг?\nИли неверное то ..."
2,2,Пушкин,NaN,0,"Когда, к мечтательному миру\nСтремясь возвышен..."
3,3,Пушкин,NaN,0,Его стихов пленительная сладость\nПройдет веко...
4,4,Пушкин,NaN,0,"Бессмертною рукой раздавленный зоил,\nПозорног..."


In [ ]:
corpus['text'][0]

'Когда сожмешь ты снова руку,\nКоторая тебе дарит\nНа скучный путь и на разлуку\nСвятую Библию Харит?\nАмур нашел ее в Цитере,\nВ архиве Шалости младой.\nПо ней молись своей Венере\nБлагочестивою душой.\nПрости, эпикуреец мой!\nОстанься век, каков ты ныне,\nЛети во мрачный Альбион!\nДа сохранят тебя в чужбине\nХристос и верный Купидон!\nНеси в чужой предел Пената,\nНо, помня прежни дни свои,\nЛюби недевственного брата,\nСтрадальца чувственной любви!'

In [ ]:
# посмотрим на данные в целом, какая длина?

# словарь (индекс: кол-во строк)
lengths = {j: i.count("\n") for i, j in zip(corpus['text'], corpus["index"])}

print("Самая маленькая поэма строк и её индекс")
print(min(lengths.values()) + 1)
print(min(lengths, key=lengths.get))

print("Самая большая поэма строк и её индекс")
print(max(lengths.values()) + 1)
print(max(lengths, key=lengths.get))

Самая маленькая поэма строк и её индекс
1
314
Самая большая поэма строк и её индекс
384
326


In [ ]:
str_lengths = {j: len(i.split()) for i, j in zip(corpus['text'], corpus["index"])}

print("Самая маленькая поэма слов и её индекс")
print(min(str_lengths.values()) + 1)
print(min(str_lengths, key=str_lengths.get))

print("Самая большая поэма слов и её индекс")
print(max(str_lengths.values()) + 1)
print(max(str_lengths, key=str_lengths.get))

Самая маленькая поэма слов и её индекс
3
314
Самая большая поэма слов и её индекс
2048
326


In [ ]:
corpus.loc[corpus['index'] == 314]
# и что это такое, есть плохие данные

,index,author,name,year,text
314,314,Пушкин,NaN,0,(ПОДРАЖАНИЕ ФРАНЦУЗСКОМУ).


In [ ]:
keys_ = [key for key, val in str_lengths.items() if val > 1000]
len(keys_)

12

In [ ]:
keys_

[296, 311, 326, 338, 380, 384, 392, 735, 811, 817, 843, 889]

In [ ]:
# удалим пока стихи где меньше 4 строк
keys_ = [key for key, val in lengths.items() if val < 3]
len(keys_)

13

In [ ]:
corpus.loc[corpus['index'] == 326]

,index,author,name,year,text
326,326,Батюшков,Странствователь и домосед,1815,"Объехав свет кругом,\nСпокойный домосед, перед..."


In [ ]:
for i in keys_:
    stroka = ((corpus.loc[corpus['index'] == i]).text.values)[0]
    print(stroka)
    print()

Аптеку позабудь ты для венков лавровых
И не мори больных, но усыпляй здоровых.

Как беспокоюсь я! три ночи,
Поверьте, глаз я не смыкал». —
«Да, слышал, слышал: в банк играл.»

Он песни пишет зло,
Как Геркулес, сатиры пишет,
Влюблен, как Буало.

(ПОДРАЖАНИЕ ФРАНЦУЗСКОМУ).

Прельщать поэзией я дара не имею
Других бы мог хвалить, тебя хвалить  не смею!

Сей камень над моей возлюбленной женой!
Ей там, мне здесь покой!

Испытанных друзей для новых забывать
Есть  цвет плоду предпочитать!

Лягушке вздумалось: сем сделаюсь с быка,
Хотя и лопну я  да мысль-то велика!

Мы не смерти боимся, но с телом расстаться нам жалко:
Так не с охотою мы старый сменяем халат.

И вещего бояна опустили
Сквозь запах роз и песни соловьев
Под тень олив, на ложе из цветов.

Жизнью земною играла она, как младенец игрушкой.
Скоро разбила ее: верно, утешилась там.

Сладкие слезы первой любви, как роса, вы иссохли!
 Нет! на бессмертных цветах в светлом раю мы блестим!

Ты переводчик, я читатель,
Ты усыпитель  я зевател

In [ ]:
# удалим все поэмы длина которых меньше 4 строк
corpus_new = corpus.copy()
corpus_new = corpus_new.drop(keys_).reset_index(drop=True)
corpus_new.shape

(1018, 5)

In [ ]:
# Как расположены экземпляры в наборе? Ответ: По порядку, сперва стихи одного автора потом другого, значит их нужно перемешать!
corpus_new

,index,author,name,year,text
0,0,Пушкин,NaN,0,"Когда сожмешь ты снова руку,\nКоторая тебе дар..."
1,1,Пушкин,NaN,0,"Тебя ль я видел, милый друг?\nИли неверное то ..."
2,2,Пушкин,NaN,0,"Когда, к мечтательному миру\nСтремясь возвышен..."
3,3,Пушкин,NaN,0,Его стихов пленительная сладость\nПройдет веко...
4,4,Пушкин,NaN,0,"Бессмертною рукой раздавленный зоил,\nПозорног..."
...,...,...,...,...,...
1013,1027,Дельвиг,66,1819,"Друзья, поверьте, не грешно\nЛюбить с вином бо..."
1014,1028,Дельвиг,А. С. ПУШКИНУ (Из Малороссии),1817,"А я ужель забыт тобою,\nМой брат по музе, мой ..."
1015,1029,Дельвиг,К ДОРИДЕ,1815,"Дорида, Дорида! любовью все дышит,\nВсе пьет н..."
1016,1030,Дельвиг,РОМАНС,1823,Вчера вакхических друзей\nЯ посетил кружок вес...


In [ ]:
# Определим функцию, чтобы вытащить общие сведения из данных
def get_info(data):
    info = pd.DataFrame(data=data.isnull().sum(), columns=["NaNs number"], index=data.columns)
    info["dtype"] = data.dtypes
    info['uniq el'] = [len(pd.unique(data.iloc[:,k])) for k in range(0, len(data.columns))]
    print(data[data.author=="Пушкин"].shape[0], " -- ",data[data.author=="Пушкин"].shape[0]/len(corpus.author))
    print(info)

In [ ]:
get_info(corpus)

323  --  0.31328806983511154
        NaNs number   dtype  uniq el
index             0   int64     1031
author            0  object        5
name            323  object      629
year              0   int64       49
text              0  object     1031


In [ ]:
print(corpus.author.unique())

['Пушкин' 'Батюшков' 'Боратынский' 'Жуковский' 'Дельвиг']


In [ ]:
# Не будем изменять исходный dataframe
from sklearn.preprocessing import LabelEncoder 

df = corpus_new.copy(deep=True)
labels_map = {'Пушкин': 1, 'Батюшков': 0, 'Боратынский': 0, 'Жуковский': 0, 'Дельвиг': 0}

y = df.author.map(labels_map) 
X = df.text

print(X.shape, y.shape)

(1018,) (1018,)


In [ ]:
print(X[0], "\n", y[0])

Когда сожмешь ты снова руку,
Которая тебе дарит
На скучный путь и на разлуку
Святую Библию Харит?
Амур нашел ее в Цитере,
В архиве Шалости младой.
По ней молись своей Венере
Благочестивою душой.
Прости, эпикуреец мой!
Останься век, каков ты ныне,
Лети во мрачный Альбион!
Да сохранят тебя в чужбине
Христос и верный Купидон!
Неси в чужой предел Пената,
Но, помня прежни дни свои,
Люби недевственного брата,
Страдальца чувственной любви! 
 1


In [ ]:
# посмотрим какие символы вообще есть в тексте помимо русских букв

def what_chars_do_we_have(dataframe):
    text = list()
    chars = "йцукенгшщзхъфывапролджэячсмитёьбю\n"
    for i in dataframe: 
        temp = i.lower()
        text_temp = "".join([ch for ch in temp if ch not in chars])
        text.append(text_temp)
    
    return text


evr = set()
tempy = what_chars_do_we_have(X)
for i in tempy:
    evr = evr | set(i)

# \xa0 - space
evr

# заментим что есть буквы латинского (английского языка), они не многочислены
# их также удалим

{' ',
 '!',
 '"',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 ':',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 '\xa0',
 '«',
 '»',
 '́',
 '—'}

In [ ]:
# Для этого исходный текст нужно разделить на слова и к каждому слову применить стэмминг
# Стоп слова исключать не будет, так как в контексте стихов они могут играть важную роль
# убрали пунктуацию, цифры

to_remove = {'!','"', '(', ')', '*', ',', '-', '.', ':', '?', '[', ']', 'a',
             'b', 'c', 'd', 'e', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 
             'r', 's', 't', 'u', 'v', 'x', '«', '»', '́', '—', "\xa0"}


def clean_text(dataframe, to_remove):
    
    text = list()
    for i in dataframe: 
        temp = i.lower()
        text_temp = "".join([ch for ch in temp if ch not in to_remove])
        final = text_temp.replace('\n', ' ')
        text.append(final)
    return text



# у нас есть список чистых текстов
X_clear = clean_text(X, to_remove)

print(len(X_clear))
print(X_clear[0])

1018
когда сожмешь ты снова руку которая тебе дарит на скучный путь и на разлуку святую библию харит амур нашел ее в цитере в архиве шалости младой по ней молись своей венере благочестивою душой прости эпикуреец мой останься век каков ты ныне лети во мрачный альбион да сохранят тебя в чужбине христос и верный купидон неси в чужой предел пената но помня прежни дни свои люби недевственного брата страдальца чувственной любви


# **НАЧАЛО РАБОТЫ С ДАННЫМИ**


In [ ]:
# модель мешка слов не подходит для реккурентных сетей, поэтому используем числовое представление + embedding слой в сети

# теперь используем встроенный токенизатор keras 
# в keras токенизатор упоряд-ет слова по частоте встречаемости, 
# поэтому последовательности в keras отличаются от нами сделанных

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, 
                      split=' ', 
                      char_level=False)

# словарь формируется на основе встречаемости слов в тексте (больше->меньше)
tokenizer.fit_on_texts(X_clear)
data = tokenizer.texts_to_sequences(X_clear)

cut_seq = []
for i in data:
    if len(i) > 1000:
        cut_seq.append(i[:1000])
    else: 
        cut_seq.append(i)

keras_pad_cut_seq = pad_sequences(cut_seq)
keras_pad_seq = pad_sequences(data)

# padding
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

# количество слов в словаре
word_count_ker = len(tokenizer.word_index)

# максимальная длина последовательности
max_seq_length = len(keras_pad_seq[0])

print(word_count_ker, max_seq_length, keras_pad_seq.shape)
keras_pad_seq
# tokenizer.index_word
# tokenizer.word_index

Found 31507 unique tokens.
31507 2047 (1018, 2047)


array([[    0,     0,     0, ...,  1323, 13132,    43],
       [    0,     0,     0, ...,     1,  1783,   329],
       [    0,     0,     0, ...,  2286,     1,  3024],
       ...,
       [    0,     0,     0, ...,   667,    17,   216],
       [    0,     0,     0, ..., 31498,   186, 31499],
       [    0,     0,     0, ..., 31506,     4,   835]], dtype=int32)

In [ ]:
# обучаем векторные представления embeddings на больших послед, учим модели на обрезанном
print(keras_pad_cut_seq.shape)
keras_pad_cut_seq

(1018, 1000)


array([[    0,     0,     0, ...,  1323, 13132,    43],
       [    0,     0,     0, ...,     1,  1783,   329],
       [    0,     0,     0, ...,  2286,     1,  3024],
       ...,
       [    0,     0,     0, ...,   667,    17,   216],
       [    0,     0,     0, ..., 31498,   186, 31499],
       [    0,     0,     0, ..., 31506,     4,   835]], dtype=int32)

In [ ]:
# заметим что последовательности разной длины, дополним их заполнителями 0, 
# внимательно следим за dtype, keras не любит object, в который по дефолту преобразуются списки

X_seq = pd.DataFrame({'texts': X_clear})

X_seq["prepr"] = X_seq.apply(lambda row: row['texts'].split(), axis=1)
keras_pad_seq[0,-100:]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,    23, 13125,     7,   263,   191,  1604,
          38,  1782,     6,  2277,   117,     1,     6,   978,  2004,
       13126,  1322,   770,   807,    36,     2,  5735,     2, 13127,
        1783,   329,    24,    76,  2586,    81,  5736, 13128,   113,
         205, 13129,    15,  1445,   131,  2587,     7,   279,   808,
          53,   979,  4398,    64, 13130,    35,     2,  2005,  4399,
           1,   280,  3016,  3017,     2,  1124,   910,  5737,     9,
        5738,  3591,    94,   167,  2278, 13131,   809,  1323, 13132,
          43], dtype=int32)

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=4a9c4370ca3e05c2f597cc3afb358999a8c6504f10ca2c4e525677e7fbbd323f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
# будем использовать word2vec и Glove, для этого необходимо привести все слова к нормальной форме
def preproc(text, morph):
    preprocessed_text = []
    for token in text:
        preprocessed_text.append(morph.parse(token)[0].normal_form)
    return preprocessed_text

import urllib.request 
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

X_seq["norm"] = X_seq.apply(lambda row: preproc(row["prepr"], morph), axis=1) 

# аналогично сделаем нормальную последовательность 

norm_tokenizer = Tokenizer(analyzer=lambda x: x,
                          lower=True, 
                          split=' ', 
                          char_level=False)

# словарь формируется на основе встречаемости слов в тексте (больше->меньше)
norm_tokenizer.fit_on_texts(X_seq["norm"])
norm_data = norm_tokenizer.texts_to_sequences(X_seq["norm"])
keras_pad_norm_seq = pad_sequences(norm_data)

# padding
norm_tokenizer.word_index['<pad>'] = 0
norm_tokenizer.index_word[0] = '<pad>'

norm_train_word_index = norm_tokenizer.word_index
print('Found %s unique tokens.' % len(norm_train_word_index))

# количество слов в словаре
word_count_norm_ker = len(norm_tokenizer.word_index)

# максимальная длина последовательности
max_norm_seq_length = len(keras_pad_norm_seq[0])

print(word_count_norm_ker, max_norm_seq_length, keras_pad_norm_seq.shape)
keras_pad_norm_seq

Found 13428 unique tokens.
13428 2047 (1018, 2047)


array([[    0,     0,     0, ...,  1023,  4406,    27],
       [    0,     0,     0, ...,     1,  1300,   466],
       [    0,     0,     0, ...,   290,     1,   323],
       ...,
       [    0,     0,     0, ...,   376,    26,    59],
       [    0,     0,     0, ...,  2296,    76, 13427],
       [    0,     0,     0, ...,   525,     5,    49]], dtype=int32)

In [ ]:
X_seq

,texts,prepr,norm
0,когда сожмешь ты снова руку которая тебе дарит...,"[когда, сожмешь, ты, снова, руку, которая, теб...","[когда, сжать, ты, снова, рука, который, ты, д..."
1,тебя ль я видел милый друг или неверное то был...,"[тебя, ль, я, видел, милый, друг, или, неверно...","[ты, ль, я, видеть, милый, друг, или, неверный..."
2,когда к мечтательному миру стремясь возвышенно...,"[когда, к, мечтательному, миру, стремясь, возв...","[когда, к, мечтательный, мир, стремиться, возв..."
3,его стихов пленительная сладость пройдет веков...,"[его, стихов, пленительная, сладость, пройдет,...","[он, стих, пленительный, сладость, пройти, век..."
4,бессмертною рукой раздавленный зоил позорного ...,"[бессмертною, рукой, раздавленный, зоил, позор...","[бессмертный, рука, раздавить, зоил, позорный,..."
...,...,...,...
1013,друзья поверьте не грешно любить с вином бокал...,"[друзья, поверьте, не, грешно, любить, с, вино...","[друг, поверьте, не, грешный, любить, с, вино,..."
1014,а я ужель забыт тобою мой брат по музе мой оре...,"[а, я, ужель, забыт, тобою, мой, брат, по, муз...","[а, я, ужель, забытый, ты, мой, брат, по, муза..."
1015,дорида дорида любовью все дышит все пьет насла...,"[дорида, дорида, любовью, все, дышит, все, пье...","[дорида, дорида, любовь, всё, дышать, всё, пит..."
1016,вчера вакхических друзей я посетил кружок весе...,"[вчера, вакхических, друзей, я, посетил, кружо...","[вчера, вакхический, друг, я, посетить, кружок..."


# **BERT С НАДСТРОЙКОЙ**

In [ ]:
! pip install transformers

In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

tqdm.pandas()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Следующий код выведет график длин предложений. Он поможет определить 
# оптимальную длину последовательности токенов, чтобы избежать разреженных векторов.


seq_len = [len(str(i).split()) for i in X]
pd.Series(seq_len).hist(bins = 50)

In [ ]:
# Для Bert не нужно проводить препроцессинг, поскольку он обучался на таких же грязных данных.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, test_size=0.15, random_state=0, shuffle=True)

batch_size = 8
max_seq_len = 512

train_text = X_train[:700].reset_index(drop=True)
train_labels = y_train[:700].reset_index(drop=True)
val_text = X_train[700:].reset_index(drop=True)
val_labels = y_train[700:].reset_index(drop=True)
test_text = X_test.reset_index(drop=True)
test_labels = y_test.reset_index(drop=True)

print(len(train_text), len(val_text), len(test_text), " = " ,len(train_text)+len(val_text)+len(test_text))

In [ ]:
# Проверяем доступность устройства CUDA
bert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
bert_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [ ]:
print(bert.config)

In [ ]:
def process_data(data_text, labels, tokenizer):

  tokens = tokenizer.batch_encode_plus(
    data_text,
    max_length = max_seq_len,
    padding = True,
    truncation = True
  )

  data_seq = torch.tensor(tokens['input_ids'])
  data_mask = torch.tensor(tokens['attention_mask'])
  data_y = torch.tensor(labels.values)

  final_data = TensorDataset(data_seq, data_mask, data_y)
  sampler = RandomSampler(final_data)
  dataloader = DataLoader(final_data, sampler = sampler, batch_size = batch_size)

  return dataloader


# делаем датасет: данные + маска + метки
train_dataloader = process_data(train_text, train_labels, bert_tokenizer)
val_dataloader = process_data(val_text, val_labels, bert_tokenizer)

In [ ]:
for param in bert.parameters():
    param.requires_grad = False

class BERT_Arch(nn.Module):
    
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768,384)
        self.fc2 = nn.Linear(384,2)
        self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self, sent_id, mask):
        _, cls_hs = self.bert(input_ids = sent_id, attention_mask = mask, return_dict = False)
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
bert_model = BERT_Arch(bert)

bert_model = bert_model.to(device)
from transformers import AdamW

optimizer = AdamW(bert_model.parameters(),
                  lr= 1e-2)


total_params = sum(p.numel() for p in bert_model.parameters())
trainable_params = sum(p.numel() for p in bert_model.parameters() if p.requires_grad)

print(f"Number of total parameters: {total_params}")
print(f"Number of trainable parameters: {trainable_params}")

Number of total parameters: 178149506
Number of trainable parameters: 296066


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
bert_model

In [ ]:
# боремся с дисбалансом классов

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)

print(class_weights)
# [0.72818312 1.59561129]

weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.to(device)
cross_entropy = nn.CrossEntropyLoss()

[0.72818312 1.59561129]


In [ ]:
def train(bert_model, train_dataloader):
    bert_model.train()
    total_loss, total_accuracy = 0, 0
    total_preds = []
    
    for step, batch in tqdm(enumerate(train_dataloader), total = len(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
        bert_model.zero_grad()
        preds = bert_model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)
        optimizer.step()
        preds = preds.detach().cpu().numpy()
        total_preds.append(preds)
        
    avg_loss = total_loss / len(train_dataloader)
    total_preds = np.concatenate(total_preds, axis = 0)
    
    return avg_loss, total_preds

In [ ]:
def evaluate(bert_model, val_dataloader):
    bert_model.eval()
    total_loss, total_accuracy = 0,0
    total_preds = []

    for step, batch in tqdm(enumerate(val_dataloader), total = len(val_dataloader)):
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        
        with torch.no_grad():
            preds = bert_model(sent_id, mask)
            loss = cross_entropy(preds, labels)
            total_loss = total_loss + loss.item()
            preds = preds.detach().cpu().numpy()
            total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)
    total_preds = np.concatenate(total_preds, axis = 0)
    return avg_loss, total_preds

In [ ]:
# Training loss: 0.564 - Validation loss: 0.593

best_valid_loss = float('inf')
epochs = 30

train_losses = []
valid_losses = []

for epoch in range(epochs):
    print('\n Epoch{:} / {:}'.format(epoch+1, epochs))
    
    train_loss, _ = train(bert_model, train_dataloader)
    valid_loss, _ = evaluate(bert_model, val_dataloader)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(bert_model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'\nTraining loss: {train_loss:.3f}')
    print(f'Validation loss: {valid_loss:.3f}')

In [ ]:
path = 'saved_weights.pt'
bert_model.load_state_dict(torch.load(path))

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# test_dataloader = process_data(test_text, test_labels, bert_tokenizer)

tokens_test = bert_tokenizer.batch_encode_plus(
    test_text,
    max_length = max_seq_len,
    padding = True,
    truncation = True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.values)

list_seq = np.array_split(test_seq, 50)
list_mask = np.array_split(test_mask, 50)


predictions = []
for num, elem in enumerate(list_seq):
    with torch.no_grad():
        preds = bert_model(elem.to(device), list_mask[num].to(device))
        predictions.append(preds.detach().cpu().numpy())

In [ ]:
PRED = pd.DataFrame({'target': test_labels.values, "text": test_text.values})

flat_preds = [item[1] for sublist in predictions for item in sublist]
flat_preds = (flat_preds - min(flat_preds)) / (max(flat_preds) - min(flat_preds))
PRED['confidence'] = flat_preds

In [ ]:
PRED['pred'] = PRED['confidence'].apply(lambda x: 1 if x > 0.54 else 0)

print(classification_report(PRED['target'], PRED['pred']))

In [ ]:
PRED

,target,text,confidence
0,0,"Твой детский вызов мне приятен,\nНо не желай м...",0.592639
1,0,"Ты прав, мой друг, ты прав хвалить ее не смей...",0.408588
2,1,"Блажен, кто в шуме городском\nМечтает об уедин...",0.783474
3,0,"Однажды Смерть послала в ад указ,\nЧтоб весь п...",0.895598
4,1,"Блажен, кто в отдаленной сени,\nВдали взыскате...",0.763405
...,...,...,...
148,1,"Счастлив, кто близ тебя, любовник упоенный,\nБ...",0.944334
149,0,"Лягушке вздумалось: сем сделаюсь с быка,\nХотя...",0.283586
150,0,"Как песенка моя понравилась Лилете,\nОна ее н...",0.576783
151,1,Старайся наблюдать различные приметы:\nПастух ...,0.887725


In [ ]:
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

def bert_score(y_test, y_prob, threshold=0.5):
    
    # метод keras.model.predict возвращает именно вероятности а не логиты (всё окей!!!)
    preds = [1 if prob > threshold else 0 for prob in y_prob]
    
    #print(preds)
    # y_pred2 = [round(y_prob[i,0]) for i in range(len(y_prob))]
    
    real = Counter(y_test)
    print('real: ', real)
    cnt = Counter(preds)
    print("preds: ", cnt)
    
    print("Confusion matrix: ")
    print(confusion_matrix(y_test, preds))
    
    #print(y_prob.shape)
    #print(y_prob[:,0])

    print("Here: ", {"ROC AUC": roc_auc_score(y_test, y_prob),
            "Balanced Accuracy": balanced_accuracy_score(y_test, preds),
            "F1": f1_score(y_test, preds)})


In [ ]:
bert_score(PRED['target'], PRED['confidence'], threshold=0.64)

# **BERT ЦЕЛИКОМ**

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [ ]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=2).to(device)
model_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [ ]:
# Данная модель принимает предложения длиной, не больше 512 токенов

# Для Bert не нужно проводить препроцессинг, поскольку он обучался на таких же грязных данных.
# данные те же самые 1018 текстов - внимательно с индексам после удаления некоторых записей в df !!!
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, test_size=0.15 , random_state=0, shuffle=True)

train_text = X_train[:700].reset_index(drop=True)
train_labels = y_train[:700].reset_index(drop=True)
val_text = X_train[700:].reset_index(drop=True)
val_labels = y_train[700:].reset_index(drop=True)
test_text = X_test.reset_index(drop=True)
test_labels = y_test.reset_index(drop=True)

print(len(train_text), len(val_text), len(test_text), " = " ,len(train_text)+len(val_text)+len(test_text))

700 165 153  =  1018


In [ ]:
max_seq_len = 512

tokens_train = model_tokenizer.batch_encode_plus(
    train_text,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = model_tokenizer.batch_encode_plus(
    test_text,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        print(idx)
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

In [ ]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [ ]:
# Ниже указаны все параметры, которые будут использоваться для обучения:

training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

# Передача в trainer предообученную модель, tokenizer, данные для обучения, данные для валидации и способ расчета метрики:
# ДАННАЯ МОДЕЛЬ ИМЕЕТ 180 МИЛЛИОН ПАРАМЕТРОВ - НА МОЁМ УСТРОЙСТВЕ ОБУЧИТЬ 
# ТАКУЮ МОДЕЛЬ - НЕВОЗМОЖНО

trainer = Trainer(model=model,
                  tokenizer = model_tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

In [ ]:
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))

# trans

In [ ]:
# на будущее 
# lets use huggingface (transformers)

!pip install transformers sentencepiece

from transformers import pipeline
# для автоматизации шагов обработки текста

classifier = pipeline("text-classification", "papluca/xlm-roberta-base-language-detection")
